In [79]:
import collections

In [93]:
# Defining Operation sets
fp_ops = {"FADD", "FSUB", "FMUL", "FDIV"}
fp_logic_ops = {"FAND", "FOR", "FXOR", "FNOT"}
int_add_ops = {"IADD", "ISUB"}
int_mul_ops = {"IMUL", "IDIV"}
int_logic_ops = {"IAND", "IOR", "IXOR", "INOT"}
store_ops = {"ST"}
load_ops = {"LD"}

# Defining Operation Latencies
op_latency = {
    "LD": 1,
    "FADD": 18,
    "FSUB": 18,
    "FMUL": 30,
    "FDIV": 40,
    "FAND": 1,
    "FOR": 1,
    "FXOR": 1,
    "FNOT": 1,
    "IADD": 6,
    "ISUB": 6,
    "IMUL": 12,
    "IDIV": 16,
    "IAND": 1,
    "IOR": 1,
    "IXOR": 1,
    "INOT": 1,
    "ST": 1
}

# Instruction class
class Instruction:
    def __init__(self, op, dest, src1=None, src2=None, address=None, index=0):
        self.op = op
        self.dest = dest
        self.src1 = src1
        self.src2 = src2
        self.address = address
        self.index = index
        # Timing info
        self.issue = None
        self.start = None
        self.complete = None
        self.write = None
        # Reservation station allocated (by name)
        self.res_station = None

    def __repr__(self):
        if self.op == "ST":
            return f"ST {self.address} {self.dest}".strip()
        else:
            parts = [self.op, self.dest]
            if self.src1:
                parts.append(self.src1)
            if self.src2:
                parts.append(self.src2)
            if self.address is not None and self.op != "ST":
                parts.append(str(self.address))
            return " ".join(parts)

# Reservation Station entry.
class ReservationStation:
    def __init__(self, name, op_type):
        self.name = name
        self.op_type = op_type
        self.busy = False
        self.op = None
        self.dest = None
        self.Vj = None
        self.Vk = None
        self.Qj = None
        self.Qk = None
        self.address = None
        self.instr = None
        self.exec_remaining = None
        # New attributes for broadcast tracking:
        self.broadcasted = False
        self.result = None

    def clear(self):
        self.busy = False
        self.op = None
        self.dest = None
        self.Vj = None
        self.Vk = None
        self.Qj = None
        self.Qk = None
        self.address = None
        self.instr = None
        self.exec_remaining = None
        self.broadcasted = False
        self.result = None

    def __repr__(self):
        if self.busy:
            return (f"[{self.name}: {self.op} dest={self.dest} "
                    f"Vj={self.Vj} Vk={self.Vk} Qj={self.Qj} Qk={self.Qk} "
                    f"Rem={self.exec_remaining}]")
        else:
            return f"[{self.name}: Free]"

import collections

# Tomasulo Simulator class.
class TomasuloSimulator:
    def __init__(self, instructions):
        self.instructions = instructions
        self.current_cycle = 1
        self.instr_queue = collections.deque(instructions)
        self.completed_instructions = []
        self.memory = [i if i < 100 else float(i-99) for i in range(200)]  # 64 BIT MEMORY INITIALISATION

        # FP register file: registers F0-F15 mapped to (value, tag)
        self.FP_registers = {f"F{i}": (0, None) for i in range(16)}
        # Integer register file: registers R0-R15 mapped to (value, tag)
        self.int_registers = {f"R{i}": (0, None) for i in range(16)}

        # Load and Store Buffers for memory operations.
        self.load_buffer = [ReservationStation(f"Load{i+1}", "LD") for i in range(2)]
        self.store_buffer = [ReservationStation(f"Store{i+1}", "ST") for i in range(2)]

        # Reservation Stations for FP arithmetic operations.
        self.faddRS = [ReservationStation(f"FAdd{i+1}", "FADD") for i in range(4)]
        self.fmulRS = [ReservationStation(f"FMul{i+1}", "FMUL") for i in range(4)]
        self.flogicRS = [ReservationStation(f"FLogic{i+1}", "FLOGIC") for i in range(2)]

        # Reservation Stations for integer operations.
        self.iaddRS = [ReservationStation(f"IAdd{i+1}", "IADD") for i in range(2)]
        self.imulRS = [ReservationStation(f"IMul{i+1}", "IMUL") for i in range(2)]
        self.ilogicRS = [ReservationStation(f"ILogic{i+1}", "ILOGIC") for i in range(2)]

    def get_all_reservation_stations(self):
        """Helper method to return all RS groups as a single list."""
        return (self.load_buffer + self.faddRS + self.fmulRS + self.flogicRS +
                self.store_buffer + self.iaddRS + self.imulRS + self.ilogicRS)

    def find_free_rs(self, op):
        """Return a free reservation station for non-memory operations."""
        if op in fp_ops:
            if op in {"FADD", "FSUB"}:
                for rs in self.faddRS:
                    if not rs.busy:
                        return rs
            elif op in {"FMUL", "FDIV"}:
                for rs in self.fmulRS:
                    if not rs.busy:
                        return rs
        elif op in fp_logic_ops:
            for rs in self.flogicRS:
                if not rs.busy:
                    return rs
        elif op in int_add_ops:
            for rs in self.iaddRS:
                if not rs.busy:
                    return rs
        elif op in int_mul_ops:
            for rs in self.imulRS:
                if not rs.busy:
                    return rs
        elif op in int_logic_ops:
            for rs in self.ilogicRS:
                if not rs.busy:
                    return rs
        return None

    def issue(self):
        if not self.instr_queue:
            return

        instr = self.instr_queue[0]
        if instr.op in load_ops:
            # Issue to a free load buffer entry.
            rs = None
            for candidate in self.load_buffer:
                if not candidate.busy:
                    rs = candidate
                    break
            if rs is None:
                return  # Stall if no free load buffer.
            instr.issue = self.current_cycle
            rs.busy = True
            rs.op = "LD"
            rs.instr = instr
            rs.address = instr.address
            # Record the destination register for display.
            rs.dest = instr.dest
            # Include extra cycle for CDB transfer.
            rs.exec_remaining = op_latency["LD"] + 1
            # Claim the destination register.
            if instr.dest.startswith("F"):
                self.FP_registers[instr.dest] = (None, rs.name)
            else:
                self.int_registers[instr.dest] = (None, rs.name)
            instr.res_station = rs.name
            self.instr_queue.popleft()
            return

        elif instr.op in store_ops:
            # Issue to a free store buffer entry.
            rs = None
            for candidate in self.store_buffer:
                if not candidate.busy:
                    rs = candidate
                    break
            if rs is None:
                return  # Stall if no free store buffer.
            instr.issue = self.current_cycle
            rs.busy = True
            rs.op = "ST"
            rs.instr = instr
            rs.address = instr.address
            # Record the source register (the register whose value is to be stored).
            rs.dest = instr.dest
            # For store, retrieve the value (or tag) from the register file.
            if instr.dest.startswith("F"):
                value, tag = self.FP_registers.get(instr.dest, (None, None))
                if tag is None:
                    rs.Vj = value
                else:
                    rs.Qj = tag
            elif instr.dest.startswith("R"):
                value, tag = self.int_registers.get(instr.dest, (None, None))
                if tag is None:
                    rs.Vj = value
                else:
                    rs.Qj = tag
            rs.exec_remaining = op_latency["ST"] + 1
            instr.res_station = rs.name
            self.instr_queue.popleft()
            return

        else:
            # For non-memory instructions, use the standard reservation stations.
            rs = self.find_free_rs(instr.op)
            if rs is None:
                return  # Stall if no free RS available.
            instr.issue = self.current_cycle
            rs.busy = True
            rs.op = instr.op
            rs.instr = instr
            instr.res_station = rs.name

            if instr.op in fp_ops or instr.op in fp_logic_ops:
                # FP arithmetic/logical (non-load)
                for operand in ['src1', 'src2']:
                    reg = getattr(instr, operand)
                    if reg is None:
                        continue
                    value, tag = self.FP_registers[reg]
                    if tag is None:
                        if operand == 'src1':
                            rs.Vj = value
                        else:
                            rs.Vk = value
                    else:
                        if operand == 'src1':
                            rs.Qj = tag
                        else:
                            rs.Qk = tag
                rs.exec_remaining = op_latency[instr.op] + 1
                # Claim the destination register.
                self.FP_registers[instr.dest] = (None, rs.name)
            else:
                # Integer operations.
                for operand in ['src1', 'src2']:
                    reg = getattr(instr, operand)
                    if reg is None:
                        continue
                    value, tag = self.int_registers[reg]
                    if tag is None:
                        if operand == 'src1':
                            rs.Vj = value
                        else:
                            rs.Vk = value
                    else:
                        if operand == 'src1':
                            rs.Qj = tag
                        else:
                            rs.Qk = tag
                rs.exec_remaining = op_latency[instr.op] + 1
                self.int_registers[instr.dest] = (None, rs.name)
            self.instr_queue.popleft()

    def execute(self):
        """Execute instructions in all reservation station groups and buffers."""
        all_groups = [self.load_buffer, self.faddRS, self.fmulRS, self.flogicRS,
                      self.store_buffer, self.iaddRS, self.imulRS, self.ilogicRS]
        for group in all_groups:
            for rs in group:
                if not rs.busy:
                    continue
                # For store instructions, wait until the operand is ready.
                if rs.op == "ST" and rs.Qj is not None:
                    continue
                # For non-memory instructions, wait until both operands are ready.
                if rs.op not in {"LD", "ST"} and (rs.Qj is not None or rs.Qk is not None):
                    continue
                # Ensure execution does not start in the same cycle as issue.
                if rs.instr.start is None and self.current_cycle > rs.instr.issue:
                    rs.instr.start = self.current_cycle
                # Only decrement if exec_remaining is greater than 0.
                if rs.instr.start is not None and rs.exec_remaining is not None and rs.exec_remaining > 0:
                    rs.exec_remaining -= 1
                    if rs.exec_remaining == 0:
                        rs.instr.complete = self.current_cycle
                        rs.instr.write = self.current_cycle + 1
                        rs.exec_remaining = 0  # Mark ready for write-back

    def compute_result(self, op, vj, vk):
        """Compute the result based on the operation and operand values."""
        if op == "IADD":
            return vj + vk
        elif op == "ISUB":
            return vj - vk
        elif op == "IMUL":
            return vj * vk
        elif op == "IDIV":
            return vj // vk if vk != 0 else 0
        elif op == "FADD":
            return vj + vk
        elif op == "FSUB":
            return vj - vk
        elif op == "FMUL":
            return vj * vk
        elif op == "FDIV":
            return vj / vk if vk != 0 else 0.0
        elif op in {"FAND", "FOR", "FXOR", "FNOT"}:
            if op == "FAND":
                return int(vj) & int(vk)
            elif op == "FOR":
                return int(vj) | int(vk)
            elif op == "FXOR":
                return int(vj) ^ int(vk)
            elif op == "FNOT":
                return ~int(vj)
        elif op in {"IAND", "IOR", "IXOR", "INOT"}:
            if op == "IAND":
                return vj & vk
            elif op == "IOR":
                return vj | vk
            elif op == "IXOR":
                return vj ^ vk
            elif op == "INOT":
                return ~vj
        return None

    def write_back(self):
        # Find all RS entries that are ready for write-back.
        ready_rs = [rs for rs in self.get_all_reservation_stations()
                    if rs.busy and rs.exec_remaining == 0 and rs.instr is not None]

        if not ready_rs:
            return

        # Select the RS with the smallest instruction index (i.e. oldest instruction).
        rs_to_write = min(ready_rs, key=lambda rs: rs.instr.index)

        # If the result hasn't been broadcasted yet, compute and broadcast it.
        if not rs_to_write.broadcasted:
            if rs_to_write.op == "LD":
                result = self.memory[rs_to_write.address] if 0 <= rs_to_write.address < len(self.memory) else None
            elif rs_to_write.op == "ST":
                result = rs_to_write.Vj
                if 0 <= rs_to_write.address < len(self.memory):
                    self.memory[rs_to_write.address] = result
            elif rs_to_write.op in {"FNOT", "INOT"}:
                result = self.compute_result(rs_to_write.op, rs_to_write.Vj, None)
            else:
                result = self.compute_result(rs_to_write.op, rs_to_write.Vj, rs_to_write.Vk)
            rs_to_write.result = result
            rs_to_write.broadcasted = True

            # Forward result to waiting RS entries.
            for other in self.get_all_reservation_stations():
                if other.Qj == rs_to_write.name:
                    other.Vj = result
                    other.Qj = None
                if other.Qk == rs_to_write.name:
                    other.Vk = result
                    other.Qk = None

        # Write back: update the appropriate register file.
        if rs_to_write.instr.dest.startswith("F"):
            reg_val, reg_tag = self.FP_registers[rs_to_write.instr.dest]
            if reg_tag == rs_to_write.name:
                self.FP_registers[rs_to_write.instr.dest] = (rs_to_write.result, None)
        elif rs_to_write.instr.dest.startswith("R"):
            reg_val, reg_tag = self.int_registers[rs_to_write.instr.dest]
            if reg_tag == rs_to_write.name:
                self.int_registers[rs_to_write.instr.dest] = (rs_to_write.result, None)
        rs_to_write.instr.write = self.current_cycle
        # Optionally mark the instruction as completed.
        self.completed_instructions.append(rs_to_write.instr)
        rs_to_write.clear()


    def broadcast_result(self, rs):
        """Broadcast the computed result to waiting RS entries.
           For LD, get the value from memory.
           For ST, perform the store operation.
           For others, compute the result.
        """
        op = rs.op
        if op == "LD":
            if rs.address < 0 or rs.address >= len(self.memory):
                output_line = f"Error: Memory address {rs.address} out of range for LD.\n"
                result = None
            else:
                result = self.memory[rs.address]
        elif op == "ST":
            result = rs.Vj
            if rs.address < 0 or rs.address >= len(self.memory):
                output_line = f"Error: Memory address {rs.address} out of range for ST.\n"
            else:
                self.memory[rs.address] = result
                output_line = f"Store: Wrote value {result} from {rs.instr.dest} to address {rs.address}.\n"
            self.out_file.write(output_line)
        else:
            if op in {"INOT", "FNOT"}:
                result = self.compute_result(op, rs.Vj, None)
            else:
                result = self.compute_result(op, rs.Vj, rs.Vk)
        # Save the computed result and mark as broadcasted.
        rs.result = result
        rs.broadcasted = True

        # Forward result to waiting RS entries (including store buffers)
        for group in [self.faddRS, self.fmulRS, self.flogicRS,
                      self.iaddRS, self.imulRS, self.ilogicRS, self.store_buffer]:
            for other_rs in group:
                if other_rs.busy:
                    if other_rs.Qj == rs.name:
                        other_rs.Vj = result
                        other_rs.Qj = None
                    if other_rs.Qk == rs.name:
                        other_rs.Vk = result
                        other_rs.Qk = None

    def simulation_complete(self):
        all_rs_free = all(not rs.busy for rs in self.get_all_reservation_stations())
        return (not self.instr_queue) and all_rs_free


    def print_register_status(self, out_file):
        out_file.write("FP Register Status:\n")
        out_file.write("Register\tValue\tStatus\n")
        for reg in sorted(self.FP_registers.keys(), key=lambda r: int(r[1:])):
            value, tag = self.FP_registers[reg]
            if tag is None:
                status = "Ready"
                val_display = value
            else:
                status = f"Waiting ({tag})"
                val_display = tag
            out_file.write(f"{reg}\t\t{val_display}\t{status}\n")
        out_file.write("\n")

        out_file.write("Integer Register Status:\n")
        out_file.write("Register\tValue\tStatus\n")
        for reg in sorted(self.int_registers.keys(), key=lambda r: int(r[1:])):
            value, tag = self.int_registers[reg]
            if tag is None:
                status = "Ready"
                val_display = value
            else:
                status = f"Waiting"
                val_display = tag
            out_file.write(f"{reg}\t\t{val_display}\t{status}\n")
        out_file.write("\n")

    def print_state(self, out_file):
        out_file.write(f"Cycle {self.current_cycle}:\n")
        out_file.write("Inst\t\tIssue\tExecute\tWrite\n")
        for instr in sorted(self.instructions, key=lambda x: x.index):
            issue = instr.issue if instr.issue is not None else ""
            complete = instr.complete if instr.complete is not None else ""
            write_field = instr.write if (instr.write is not None and self.current_cycle >= instr.write) else ""
            out_file.write(f"{instr}\t{issue}\t{complete}\t\t{write_field}\n")
        out_file.write("\nReservation Stations and Buffers:\n")
        all_groups = [self.load_buffer, self.faddRS, self.fmulRS, self.flogicRS,
                      self.store_buffer, self.iaddRS, self.imulRS, self.ilogicRS]
        for group in all_groups:
            for rs in group:
                out_file.write(str(rs) + "\n")
        self.print_register_status(out_file)
        out_file.write("\nMemory Snapshot (first 10 locations):\n")
        out_file.write(str(self.memory[:10]) + "\n")
        out_file.write("-" * 50 + "\n")

    def run(self):
        with open("simulation_output.txt", "w") as self.out_file:
            while not self.simulation_complete():
                self.write_back()
                self.issue()
                self.execute()
                self.print_state(self.out_file)
                self.current_cycle += 1
                if self.current_cycle > 1000:
                    self.out_file.write("Cycle limit reached. Exiting simulation.\n")
                    break

def parse_instruction_line(line, index):
    tokens = line.strip().split()
    if not tokens:
        return None
    op = tokens[0]
    if op == "LD":
        if len(tokens) < 3:
            raise ValueError("Invalid LD instruction format")
        dest = tokens[1]
        address = int(tokens[2])
        return Instruction(op, dest, address=address, index=index)
    elif op == "ST":
        if len(tokens) < 3:
            raise ValueError("Invalid ST instruction format")
        address = int(tokens[1])
        dest = tokens[2]
        return Instruction(op, dest, address=address, index=index)
    else:
        if len(tokens) < 4:
            raise ValueError("Invalid instruction format")
        dest = tokens[1]
        src1 = tokens[2]
        src2 = tokens[3]
        return Instruction(op, dest, src1, src2, index=index)

def load_instructions_from_file(filename):
    instructions = []
    with open(filename, "r") as file:
        index = 0
        for line in file:
            if line.strip() == "" or line.strip().startswith("#"):
                continue
            instr = parse_instruction_line(line, index)
            if instr:
                instructions.append(instr)
                index += 1
    return instructions

In [94]:
if __name__ == "__main__":
    test_file = "test.txt"
    instructions = load_instructions_from_file(test_file)
    sim = TomasuloSimulator(instructions)
    sim.run()